In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm

In [ ]:
class pricing_formula:
    def __init__(self, var):
        self.S0, self.r, self.q, self.sigma, self.T, self.K1, self.K2, self.K3, self.K4 = var
        self.cal_d()
        #self.print_d()
        self.cal_Nd()
        #self.print_Nd()
        self.cal_E()
        #self.print_E()

    def cal_d(self):
        self.d = []
        self.d.append(( np.log(self.S0/self.K1) + (self.r-self.q+(self.sigma**2)/2)*self.T )/ (self.sigma*np.sqrt(self.T)))
        self.d.append(( np.log(self.S0/self.K2) + (self.r-self.q+(self.sigma**2)/2)*self.T )/ (self.sigma*np.sqrt(self.T)))
        self.d.append(( np.log(self.S0/self.K1) + (self.r-self.q-(self.sigma**2)/2)*self.T )/ (self.sigma*np.sqrt(self.T)))
        self.d.append(( np.log(self.S0/self.K2) + (self.r-self.q-(self.sigma**2)/2)*self.T )/ (self.sigma*np.sqrt(self.T)))
        self.d.append(( np.log(self.S0/self.K3) + (self.r-self.q-(self.sigma**2)/2)*self.T )/ (self.sigma*np.sqrt(self.T)))
        self.d.append(( np.log(self.S0/self.K4) + (self.r-self.q-(self.sigma**2)/2)*self.T )/ (self.sigma*np.sqrt(self.T)))
        self.d.append(( np.log(self.S0/self.K3) + (self.r-self.q+(self.sigma**2)/2)*self.T )/ (self.sigma*np.sqrt(self.T)))
        self.d.append(( np.log(self.S0/self.K4) + (self.r-self.q+(self.sigma**2)/2)*self.T )/ (self.sigma*np.sqrt(self.T)))

    def print_d(self):
        for i in range(8):
            print("d"+str(i+1)+" = ", self.d[i])

    def cal_Nd(self):
        self.Nd = []
        for i in range(8):
            self.Nd.append(norm.cdf(self.d[i]))

    def print_Nd(self):
        for i in range(8):
            print("N(d"+str(i+1)+") = ", self.Nd[i])

    def cal_E(self):
        self.E1 = self.S0 * np.exp((self.r-self.q)*self.T) * ((self.Nd[0] - self.Nd[1])) - self.K1*(self.Nd[2] - self.Nd[3])
        self.E2 = (self.K2-self.K1) * (self.Nd[3] - self.Nd[4])
        self.E3 = ((self.K2-self.K1)/(self.K3-self.K4)) * ( self.S0 * np.exp((self.r-self.q)*self.T) * (self.Nd[6]-self.Nd[7]) - self.K4 * (self.Nd[4]-self.Nd[5]))

    def print_E(self):
        print("E1 = ", self.E1)
        print("E2 = ", self.E2)
        print("E3 = ", self.E3)

    def cal_price(self):
        self.c = np.exp(-self.r * self.T) * (self.E1 + self.E2 + self.E3)
        return self.c

    def Monte_Carlo(self):
        self.mean = np.log(self.S0) + (self.r-self.q-(self.sigma**2)/2) * self.T
        self.std = self.sigma * np.sqrt(self.T)

        self.mean_array = np.zeros(20)
        for i in range(20):
            sample = np.random.normal(self.mean, self.std, 10000)
            df = pd.DataFrame(sample, columns = ['ln(S_T)'])
            df['S_T'] = np.exp(df)
            df['Payoff1'] = list(map(lambda x: x-self.K1 if self.K1 <= x < self.K2 else 0, df['S_T']))
            df['Payoff2'] = list(map(lambda x: self.K2-self.K1 if (self.K2 <= x < self.K3) else 0, df['S_T']))
            df['Payoff3'] = list(map(lambda x: (self.K2-self.K1)/(self.K3-self.K4)*(x-self.K4) if self.K3 <= x < self.K4 else 0, df['S_T']))
            df2= pd.concat([df['Payoff1'],df['Payoff2'],df['Payoff3']], axis=1)
            df['Payoff'] = df2.max(axis=1)
            df['Value'] = df['Payoff'] * np.exp(-self.r * self.T)
            mean = df['Value'].mean()
            self.mean_array[i] = mean

        mean = self.mean_array.mean()
        std = self.mean_array.std()
        #print("Confidence interval = [", mean-2*std, ",", mean+2*std,"]")
        return  mean-2*std,  mean+2*std

In [ ]:
order = ['S0', 'r', 'q', 'sigma', 'T', 'K1','K2', 'K3', 'K4']
var = []
for i in range(len(order)):
    var.append(float(input("Please type " + order[i])))

Please type S0 50
Please type r 0.04
Please type q 0.01
Please type sigma 0.3
Please type T 0.5
Please type K1 45
Please type K2 48
Please type K3 55
Please type K4 60


In [ ]:
model = pricing_formula(var)
price = model.cal_price()
print(price)

1.1014989375333561


In [ ]:
lb, ub = model.Monte_Carlo()
print("Confidence interval = [", lb, ",", ub,"]")

Confidence interval = [ 1.0826858221723825 , 1.1148629779647024 ]


In [ ]:
test = []
for i in range(500):
    test.append(model.Monte_Carlo())

In [ ]:
isin = 0
for i in range(500):
    if test[i][0] <= price <= test[i][1]:
        isin += 1
print(isin)

500
